# Search spaces in NASLib

Neural Architecure Search consists of 3 building blocks:
- Search space (cell, hierarchical, ...)
- Optimization
- Evaluation

Here we will conver the first part: Search spaces.

## Cell search space

The predominantway of representing NAS search spaces is the directed acyclic graph (DAG). Here, edges are operations (e.g. 3x3conv) and nodes are feature maps (i.e. Tensors).

The optimizer searches for an optimal cell structre (i.e. the DAG) which is the building block for the network. How these building blocks then are aranged and wired is not part of the search space and usually defined manually.

Before starting we need to import the relevant classes.

`EdgeOpGraph` represents the DAG object for our search space.

Additionally we need to import the operations available for the edges (called "Primitives")

In [5]:
from naslib.search_spaces.core import EdgeOpGraph
from naslib.search_spaces.core.primitives import ReLUConvBN, Stem, Identity

Next we can inspect the DARTS cell search space living in `search_spaces/darts/darts/graph.py`.

Let's create a class which encapsulates our cell. This will be used as building block for the makro architecture.

In [ ]:
class ToyCell(EdgeOpGraph):
    pass

Each cell needs some information in the beginning

- `primitives`: The operations which are available on the edges
- `channels`: The number of output channels for convolutional kernels

In [4]:
def __init__(self, primitives, channels, *args, **kwargs):
    self.primitives = primitives
    self.channels = channels
    super(Cell, self).__init__(*args, **kwargs)

In [3]:

    

    def _build_graph(self):
        # Input Nodes: Previous / Previous-Previous cell
        preprocessing0 = FactorizedReduce(self.C_prev_prev, self.C, affine=False) \
            if self.reduction_prev else ReLUConvBN(self.C_prev_prev, self.C, 1, 1, 0, affine=False)
        preprocessing1 = ReLUConvBN(self.C_prev, self.C, 1, 1, 0, affine=False)

        self.add_node(0, type='input', preprocessing=preprocessing0, desc='previous-previous')
        self.add_node(1, type='input', preprocessing=preprocessing1, desc='previous')

        # 4 intermediate nodes
        self.add_node(2, type='inter', comb_op='sum')
        self.add_node(3, type='inter', comb_op='sum')
        self.add_node(4, type='inter', comb_op='sum')
        self.add_node(5, type='inter', comb_op='sum')

        # Output node
        self.add_node(6, type='output', comb_op='cat_channels')

        # Edges: input-inter and inter-inter
        for to_node in self.inter_nodes():
            for from_node in range(to_node):
                stride = 2 if self.cell_type == 'reduction' and from_node < 2 else 1
                self.add_edge(
                    from_node, to_node, op=None, op_choices=self.primitives,
                    op_kwargs={'C': self.C, 'stride': stride, 'out_node_op': 'sum', 'ops_dict': self.ops_dict,
                               'affine': False},
                    to_node=to_node, from_node=from_node)

        # Edges: inter-output
        self.add_edge(2, 6, op=Identity())
        self.add_edge(3, 6, op=Identity())
        self.add_edge(4, 6, op=Identity())
        self.add_edge(5, 6, op=Identity())